### Čia duomenys paruošiami iš netvarkingo csv, gauto iš perpare_nlp.py ir pradedamas modeliavimas

In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder

In [5]:
columns = ['Sub_Category_1', 'Sub_Category_2', 'ner_dict', 'embedding']

df = pd.read_csv(
    'prepared_df.csv',
).dropna()[columns]
df.head()

/tmp/ipykernel_59285/1440718678.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,Sub_Category_1,Sub_Category_2,ner_dict,embedding
0,Namams,Tekstilė,"{'numeric': 1, 'currency': 1}",[ 0.03761789 0.42211768 2.1626077 2.754687...
1,Moterims,Avalynė,"{'numeric': 1, 'currency': 0}",[-1.9406556 1.4364058 1.5963019 1.662026...
2,Namams,Interjero akcentai,"{'numeric': 6, 'currency': 2}",[ 0.4350617 -0.25477087 1.1061407 2.625409...
3,Moterims,Drabužiai,"{'numeric': 0, 'currency': 0, 'LOC': 1}",[ 2.1903796e+00 7.8882933e-01 6.3675702e-01 ...
4,Moterims,Drabužiai,"{'numeric': 0, 'currency': 0}",[ 6.28135860e-01 -1.56554127e+00 -1.06804979e+...


In [13]:
EMBED_SIZE = 96

df['embedding'] = df['embedding']\
    .apply(lambda x: x.strip('[]'))\
    .apply(lambda x: np.fromstring(x, sep=' '))

In [14]:
for i in range(EMBED_SIZE):
    df[f'embedding{i}'] = df['embedding'].apply(lambda x: x[i])
df.drop(columns='embedding', inplace=True)

In [15]:
import ast

df_ner = pd.json_normalize(
    df['ner_dict'].apply(ast.literal_eval)
)

df = pd.concat([
    df.drop(columns=['ner_dict']),
    df_ner,
], axis='columns')

In [16]:
cat1_encoder = LabelEncoder()
df.insert( 0, 'cat1', cat1_encoder.fit_transform(df['Sub_Category_1']) )

cat2_encoder = LabelEncoder()
df.insert( 1, 'cat2', cat2_encoder.fit_transform(df['Sub_Category_2']) )

/tmp/ipykernel_110639/2478584859.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert( 1, 'cat2', cat2_encoder.fit_transform(df['Sub_Category_2']) )


In [17]:
df.drop(columns = ['Sub_Category_1', 'Sub_Category_2'], inplace = True)

,cat1,cat2,embedding0,embedding1,embedding2,embedding3,embedding4,embedding5,embedding6,embedding7,...,FAC,PER,ORDINAL,QUANTITY,WORK_OF_ART,MISC,DATE,LAW,LANGUAGE,EVENT
0,6,43,0.037618,0.422118,2.162608,2.754688,0.898877,-2.683462,-0.694830,0.942933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,4,-1.940656,1.436406,1.596302,1.662027,-0.080104,-0.884043,-0.170395,-0.179730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,22,0.435062,-0.254771,1.106141,2.625409,1.690074,-1.592022,0.456091,0.274041,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,16,2.190380,0.788829,0.636757,1.074797,0.693524,-0.940999,-1.572375,-0.295537,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,16,0.628136,-1.565541,-1.068050,-0.748494,5.129196,-1.529307,2.178699,-0.119058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.to_csv('final_df.csv')

In [23]:
np.save('classes1.npy', cat1_encoder.classes_)
np.save('classes2.npy', cat2_encoder.classes_)